<a href="https://colab.research.google.com/github/subodha1980/COVID-19/blob/master/Search_%26_Optimization_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import math
import csv
from operator import itemgetter

In [4]:
B1 = [-1   ,-1]
B2 = [56325, 9]
BS_LOC = [B1,B2]
HOP_DELAY =50
ALPHA = 2.0 # Strength = ALPHA * speed/hops

BS_REACH = 0
RATE = 1
HOPS = 2
NEXT = 3
STRENGTH = 4

V_INDEX = 0
X_INDEX = 1
Y_INDEX = 2

NO_OF_VEHICLES = 100

In [5]:
rates = [10,8,6,4,2,1,0]
dist_bins = [500,1000,2000,3000,4000,6000,999999]
def get_data_rate(dist):
  bin_id = np.digitize(dist,dist_bins) # get ID of the bin
  return rates[bin_id]

In [6]:
def get_distance(p1,p2): return math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)

In [7]:
def get_zero_List(k): return [[0 for col in range(k)] for row in range(k)]

In [8]:
file_name = '/content/drive/MyDrive/Colab Notebooks/Assignment/data_file.csv' # you can mount a file in the google drive and copy the path too

def read_file():
  with open(file_name, 'rt') as f:
      csv_reader = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC) # reading values as values (not strings)
      vehicle_data = []
      for line in csv_reader:
        vehicle_data.append([int(line[V_INDEX]),line[X_INDEX],line[Y_INDEX]])
  return vehicle_data

In [9]:
def get_strength(speed,hops):
  return (ALPHA*speed/hops)

In [10]:
def update_strength_line(bs,next,speed,line):
  #[which BS, next node, rate, hops, strength]
  line[bs][BS_REACH] = bs
  line[bs][RATE] = speed
  line[bs][HOPS] = 1
  line[bs][NEXT] = next
  line[bs][STRENGTH] = get_strength(speed,1)

  # if strength of this BS is greater than the recorded strength
  if (line[bs][STRENGTH]>line[0][STRENGTH]):
    line[0][BS_REACH] = bs
    line[0][RATE] = speed
    line[0][HOPS] = 1
    line[0][NEXT] = next
    line[0][STRENGTH] = line[bs][STRENGTH]

In [11]:
def init_neighbors():
      vehicle_data_list = read_file()

      BS1_ID = len(vehicle_data_list) + 1  # ID of BS 1
      BS2_ID = BS1_ID + 1           # ID of BS 2

      rate_list = get_zero_List(len(vehicle_data_list)+2) # rates of each hop, (+2) for two BSs
      neighbor_list= []
      strength_list = []
      for line in vehicle_data_list:
        neighbor_line = []
        #neighbor_line.append(line[V_INDEX])
        P_new = [line[X_INDEX],line[Y_INDEX]] #select one vehicle

        for row in vehicle_data_list:

            if (line[V_INDEX]!=row[V_INDEX]): # itself cannot be a neighbour
                Q_new = [row[X_INDEX],row[Y_INDEX]] #select neighbor vehicle
                rate = get_data_rate(get_distance(Q_new,P_new)) # get the rate based on the distance
                if (rate>0):neighbor_line.append([int(row[V_INDEX]),rate])
                rate_list[line[V_INDEX]-1][row[V_INDEX]-1] = rate

        strength_line = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0],line[V_INDEX]]
        rate = get_data_rate(get_distance(B1,P_new))
        rate_list[line[V_INDEX]-1][BS1_ID-1] = rate
        rate_list[BS1_ID-1][line[V_INDEX]-1] = rate
        if (rate>0):
          update_strength_line(1,BS1_ID,rate,strength_line) #Store connection details to BS1
          neighbor_line.append([BS1_ID,rate]) # Store BS1 as a neighbor

        rate = get_data_rate(get_distance(B2,P_new))
        rate_list[line[V_INDEX]-1][BS2_ID-1] = rate
        rate_list[BS2_ID-1][line[V_INDEX]-1] = rate
        if (rate>0):
          update_strength_line(2,BS2_ID,rate,strength_line) #Store connection details to BS1
          neighbor_line.append([BS2_ID,rate]) # Store BS2 as a neighbor

        strength_list.append(strength_line)
        #print(strength_line[3], neighbor_line)

        neighbor_list.append(neighbor_line)

      #print(rate_list)
      new_strength_list = list( map(itemgetter(0), strength_list ))
      return neighbor_list,rate_list,new_strength_list

In [12]:
def run_iteration(strength_list,neighbor_list):
  new_strength_list = []
  v_id = 1;
  no_of_vehicles = len(neighbor_list) # length of the list
  for neighbor_row in neighbor_list: # go throuhg neighbout list of each node

    connected_list = []
    for neighbor_node in neighbor_row:  # go though neighbor list of particular node
       n_id = neighbor_node[V_INDEX] # neighbor currently working with

       if (n_id>no_of_vehicles): break # neighbor cannot be outside the limits

       if (strength_list[n_id-1][BS_REACH] > 0): # BS connected?
          rate = min(neighbor_node[RATE],strength_list[n_id-1][RATE])     # new rate if go through this path
          hops = strength_list[n_id-1][HOPS]+1                            # new  hopd if go through this path
          next_node = n_id                                                # next node to connect to
          strngth = get_strength(rate,hops)                               # strength
          connect_bs = strength_list[n_id-1][BS_REACH]                    # which BS to connect to
          connected_list.append([connect_bs,rate,hops,next_node,strngth])

    connected_list=sorted(connected_list, key=itemgetter(STRENGTH),reverse=True)  # sorting the list w.r.t strength
    #print(connected_list)
    if (len(connected_list)>0):
        selected_node = connected_list[0]
        if (strength_list[v_id-1][STRENGTH]<selected_node[STRENGTH]):
            new_strength_list.append(selected_node)
        else:
            new_strength_list.append(strength_list[v_id-1])
    else:
        new_strength_list.append([0,0,0,0,0])
    v_id += 1 # next behicle
  return new_strength_list

In [13]:
def check_for_connectivity(strength_list,d_list):

  connected_list = np.array(list (map(itemgetter(BS_REACH), strength_list )))
  tot_connected_boolean = connected_list > 0
  tot_connected = sum(tot_connected_boolean.astype(int))

  avg_rate = sum(list( map(itemgetter(RATE), strength_list )))/NO_OF_VEHICLES
  avg_hops = sum(list( map(itemgetter(HOPS), strength_list )))/NO_OF_VEHICLES
  d_list.append([avg_rate,avg_hops,tot_connected])

In [17]:
def get_path(strength_list,node):
  path_line = []
  from_node = node
  while (node <= NO_OF_VEHICLES):
    node = strength_list[node-1][NEXT]
    speed = rate[from_node-1][node-1]
    path_line.append([from_node,node,speed])

    print(from_node,'-->',node,':',speed)
    from_node = node
  return path_line


In [15]:
(neighbor,rate,strength) = init_neighbors()
data_list = []
check_for_connectivity(strength,data_list)

for iteration in range(1,20):
  strength = run_iteration(strength,neighbor)
  check_for_connectivity(strength,data_list)
  print(strength)

print(list( map(itemgetter(0), data_list)))
print(list( map(itemgetter(1), data_list)))
print(list( map(itemgetter(2), data_list)))

[[1, 10, 1, 101, 20.0], [1, 6, 1, 101, 12.0], [1, 6, 2, 2, 6.0], [1, 6, 2, 52, 6.0], [1, 4, 2, 2, 4.0], [1, 4, 2, 2, 4.0], [1, 4, 2, 53, 4.0], [1, 4, 2, 53, 4.0], [1, 2, 2, 3, 2.0], [1, 1, 2, 3, 1.0], [1, 1, 2, 4, 1.0], [1, 1, 2, 4, 1.0], [1, 1, 2, 5, 1.0], [1, 1, 2, 5, 1.0], [1, 1, 2, 8, 1.0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [2, 1, 2, 50, 1.0], [2, 1, 2, 50, 1.0], [2, 1, 2, 50, 1.0], [2, 1, 2, 50, 1.0], [2, 2, 2, 95, 2.0], [2, 1, 1, 102, 2.0], [1, 10, 1, 101, 20.0], [1, 6, 1, 101, 12.0], [1, 4, 1, 101, 8.0], [1, 4, 2, 52, 4.0], [1, 

In [18]:
get_path(strength,61)

61 --> 59 : 4
59 --> 10 : 4
10 --> 8 : 4
8 --> 53 : 4
53 --> 101 : 4


[[61, 59, 4], [59, 10, 4], [10, 8, 4], [8, 53, 4], [53, 101, 4]]

In [ ]:
def _run_iteration(strength_list):
  new_rout_table = []
  v_id = 1;
  #print(strength_list)
  for neighbor_row in neighbor: # go throuhg neighbout list of each node

    connected_list = []
    for neighbor_node in neighbor_row:  # go though neighbor list of particular node
       n_id = neighbor_node[V_INDEX] # neighbor currently working with
       if (n_id>100): break

       if (strength_list[n_id-1][0][BS_REACH] > 0): # BS connected?
          rate = min(neighbor_node[RATE],strength_list[n_id-1][0][RATE])     # new rate if go through this path
          hops = strength_list[n_id-1][0][HOPS]+1  # new  hopd if go through this path
          next_node = n_id
          strngth = get_strength(rate,hops)
          connect_bs = strength_list[n_id-1][0][BS_REACH]
          connected_list.append([connect_bs,rate,hops,next_node,strngth])
    connected_list=sorted(connected_list, key=itemgetter(STRENGTH),reverse=True)
    #print(connected_list)
    if (len(connected_list)>0):
      selected_node = connected_list[0]
      if (strength_list[v_id-1][0][STRENGTH]<selected_node[STRENGTH]):
          strength_list[v_id-1][0] = selected_node
    v_id += 1 # next behicle
  return strength_list

13.0